In [0]:
# Enabling Schema Evolution 
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled",True)

# A Function to take File Paths as Input and convert all those files into views after reading them
def create_temp_view (input_map, frmt):
  for key in input_map:
    spark.read.format(frmt).option("header",True).load(input_map.get(key)).createOrReplaceTempview(key)

In [0]:
v_file_date="2021-12-15"
raw_folder_path="/mnt/mavendataguy/input/ecom_growth"
files_path= {"products":f"{raw_folder_path}/{v_file_date}/products", "sessions.csv":f"{raw_folder_path}/{v_file_date}/sessions.csv"}
create_temp_view (files_path, "csv")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
<command-4495866514629542> in <module>
      2 raw_folder_path="/mnt/mavendataguy/input/ecom_growth"
      3 files_path= {"products":f"{raw_folder_path}/{v_file_date}/products", "sessions.csv":f"{raw_folder_path}/{v_file_date}/sessions.csv"}
----> 4 create_temp_view (files_path, "csv")

<command-4495866514629539> in create_temp_view(input_map, frmt)
      5 def create_temp_view (input_map, frmt):
      6   for key in input_map:
----> 7     spark.read.format(frmt).option("header",True).load(input_map.get(key)).createOrReplaceTempview(key)

/databricks/spark/python/pyspark/sql/readwriter.py in load(self, path, format, schema, **options)
    202         self.options(**options)
    203         if isinstance(path, str):
--> 204             return self._df(self._jreader.load(path))
    205         elif path is not None:
    206             if

In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn("ingestion_date", current_timestamp())
  return output_df

In [0]:
def read_file(path,date,partition_column):
  df= spark.read.format("parquet").load(f"{path}")#.filter(partition_column='2020-01-01')
  #return {"return_df":df}
  return df

In [0]:
def re_arrange_partition_column(input_df, partition_column):
  column_list = []
  for column_name in input_df.schema.names:
    if column_name != partition_column:
      column_list.append(column_name)
  column_list.append(partition_column)
  output_df = input_df.select(column_list)
  return output_df

In [0]:
def overwrite_partition(input_df, db_name, table_name, partition_column):
  output_df = re_arrange_partition_column(input_df, partition_column)
  spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
  if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
    output_df.write.mode("overwrite").insertInto(f"{db_name}.{table_name}")
  else:
    output_df.write.mode("overwrite").partitionBy(partition_column).format("parquet").saveAsTable(f"{db_name}.{table_name}")

In [0]:
def df_column_to_list(input_df, column_name):
  df_row_list = input_df.select(column_name) \
                        .distinct() \
                        .collect()
  
  column_value_list = [row[column_name] for row in df_row_list]
  return column_value_list

In [0]:
def merge_delta_data(input_df, db_name, table_name, folder_path, merge_condition, partition_column):
  spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning","true")

  from delta.tables import DeltaTable
  if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
    deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")
    deltaTable.alias("tgt").merge(
        input_df.alias("src"),
        merge_condition) \
      .whenMatchedUpdateAll()\
      .whenNotMatchedInsertAll()\
      .execute()
  else:
    input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")

In [0]:
#input_df.write.mode("overwrite")..option("overwriteSchema", "true").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")